In [3]:
from transformers import pipeline, AutoTokenizer
import torch

torch.manual_seed(0)
model = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cuda:0


# Load the dataset

In [4]:
from datasets import load_dataset
def filter_inadequada(example):
    return example["INADEQUADA"] == 0

ds = load_dataset("higopires/RePro-categories-multilabel")
print(len(ds["train"]))
#remove INADEQUADA examples
ds = ds.filter(filter_inadequada)
#remove only keep the firs 1000 examples
#ds["train"] = ds["train"].filter(lambda example, idx: idx < 20, with_indices=True)
#ds["test"] = ds["test"].filter(lambda example, idx: idx < 5, with_indices=True)
#ds["validation"] = ds["validation"].filter(lambda example, idx: idx < 5, with_indices=True)
print(len(ds["train"]))

8002
7674


# Few-shot

In [3]:
def run_prompt(prompt):
    prompt = """Classifique o texto fornecido com uma ou mais das seguintes características: ENTREGA, OUTROS, PRODUTO, CONDICOESDERECEBIMENTO, ANUNCIO. Forneça a resposta como uma lista de características separadas por vírgulas.

    ### Exemplos:

    Texto: "A entrega foi super rápida, chegou em 2 dias!"
    Características: ENTREGA

    Texto: "O smartphone tem uma câmera incrível e a bateria dura o dia todo."
    Características: PRODUTO

    Texto: "O anúncio mencionava frete grátis, mas fui cobrado no final."
    Características: ANUNCIO, ENTREGA

    Texto: "Não gostei da experiência geral, muitas interrupções."
    Características: OUTROS

    Texto: "O produto chegou em boas condições e a entrega foi rapida."
    Características: ENTREGA, PRODUTO, CONDICOESDERECEBIMENTO

    ### Novo Texto:

    Texto: "{review_text}"
    Características:
    """

    # Example usage
    review_text_example = prompt
    formatted_prompt = prompt.format(review_text=review_text_example)

    sequences = pipe(
        formatted_prompt,
        max_new_tokens=50,
        return_full_text = False,
    )
    result = [0,0,0,0,0]
    for seq in sequences:
        generatedText = seq["generated_text"]
        if "ENTREGA" in generatedText:
            result[0]=1
        if "OUTROS" in generatedText:
            result[1]=1
        if "PRODUTO" in generatedText:
            result[2]=1
        if "CONDICOESDERECEBIMENTO" in generatedText:
            result[3]=1
        if "ANUNCIO" in generatedText:
            result[4]=1
    return result

    

In [4]:
Predictions = []
GroundTruth = []
for i in range(len(ds["test"])):
    if i % 100 == 0:
        print(f"Processing example {i} of {len(ds['test'])}")
    sample = ds["test"][i]
    review_text = ["review_text"]
    result = run_prompt(review_text)
    labels = [sample["ENTREGA"],sample["OUTROS"],sample["PRODUTO"],sample["CONDICOESDERECEBIMENTO"],sample["ANUNCIO"]]
    #print(f"Result: {result}")
    #print(f"Labels: {labels}")
    Predictions.append(result)
    GroundTruth.append(labels)

Processing example 0 of 966


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processing example 100 of 966
Processing example 200 of 966
Processing example 300 of 966
Processing example 400 of 966
Processing example 500 of 966
Processing example 600 of 966
Processing example 700 of 966
Processing example 800 of 966
Processing example 900 of 966


In [5]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
def compute_metrics(predictions,labels):
    # Calculate sample-wise F1 score
    f1 = f1_score(labels, predictions, average='micro', zero_division=0)
    precision = precision_score(labels, predictions, average='micro', zero_division=0)

    recall = recall_score(labels, predictions, average='micro', zero_division=0)
    return {'f1_micro': float(f1), 'precision': float(precision), 'recall': float(recall)}
print(compute_metrics(Predictions,GroundTruth))

{'f1_micro': 0.4788048552754435, 'precision': 0.33481326717158527, 'recall': 0.8401048492791612}


# Zero-shot

In [5]:
def run_prompt(prompt):
    prompt = """Classifique o texto fornecido com uma ou mais das seguintes características: ENTREGA, OUTROS, PRODUTO, CONDICOESDERECEBIMENTO, ANUNCIO. Forneça a resposta como uma lista de características separadas por vírgulas.

    Texto: "{review_text}"
    Características:
    """

    # Example usage
    review_text_example = prompt
    formatted_prompt = prompt.format(review_text=review_text_example)

    sequences = pipe(
        formatted_prompt,
        max_new_tokens=50,
        return_full_text = False,
    )
    result = [0,0,0,0,0]
    for seq in sequences:
        generatedText = seq["generated_text"]
        if "ENTREGA" in generatedText:
            result[0]=1
        if "OUTROS" in generatedText:
            result[1]=1
        if "PRODUTO" in generatedText:
            result[2]=1
        if "CONDICOESDERECEBIMENTO" in generatedText:
            result[3]=1
        if "ANUNCIO" in generatedText:
            result[4]=1
    return result

    

In [ ]:
Predictions = []
GroundTruth = []
for i in range(len(ds["test"])):
    if i % 100 == 0:
        print(f"Processing example {i} of {len(ds['test'])}")
    sample = ds["test"][i]
    review_text = ["review_text"]
    result = run_prompt(review_text)
    labels = [sample["ENTREGA"],sample["OUTROS"],sample["PRODUTO"],sample["CONDICOESDERECEBIMENTO"],sample["ANUNCIO"]]
    #print(f"Result: {result}")
    #print(f"Labels: {labels}")
    Predictions.append(result)
    GroundTruth.append(labels)
def compute_metrics(predictions,labels):
    # Calculate sample-wise F1 score
    f1 = f1_score(labels, predictions, average='micro', zero_division=0)
    precision = precision_score(labels, predictions, average='micro', zero_division=0)

    recall = recall_score(labels, predictions, average='micro', zero_division=0)
    return {'f1_micro': float(f1), 'precision': float(precision), 'recall': float(recall)}
print(compute_metrics(Predictions,GroundTruth))

Processing example 0 of 966


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processing example 100 of 966
Processing example 200 of 966
Processing example 300 of 966
Processing example 400 of 966
Processing example 500 of 966
Processing example 600 of 966
Processing example 700 of 966
Processing example 800 of 966
Processing example 900 of 966


NameError: name 'compute_metrics' is not defined

In [8]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
def compute_metrics(predictions,labels):
    # Calculate sample-wise F1 score
    f1 = f1_score(labels, predictions, average='micro', zero_division=0)
    precision = precision_score(labels, predictions, average='micro', zero_division=0)

    recall = recall_score(labels, predictions, average='micro', zero_division=0)
    return {'f1_micro': float(f1), 'precision': float(precision), 'recall': float(recall)}
print(compute_metrics(Predictions,GroundTruth))

{'f1_micro': 0.4792578330124279, 'precision': 0.3269644136613327, 'recall': 0.8971166448230669}
